# PYTORCH CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/torch_cnn_classifier.ipynb torch_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
# DATASET = 'no_dataset'
DATASET = 'human_ensembl_regulatory'
# DATASET = 'human_enhancers_ensembl'
VERSION = 0
BATCH_SIZE = 32
EPOCHS = 1

In [2]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_ensembl_regulatory 0 32 1


In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from pathlib import Path

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from genomic_benchmarks.models.torch import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config, VARIABLE_LENGTH_DATASETS
from genomic_benchmarks.data_check import is_downloaded, info

## Config

In [ ]:
USE_PADDING = DATASET in VARIABLE_LENGTH_DATASETS
    
config = {
    "dataset": DATASET,
    "dataset_version": VERSION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "use_padding": USE_PADDING,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 3,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [ ]:
# SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
# CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
# print(CLASSES)

# train_dset = tf.keras.preprocessing.text_dataset_from_directory(
#     SEQ_PATH / 'train',
#     batch_size=BATCH_SIZE,
#     class_names=CLASSES)

# info(config["dataset"])
info(config["dataset"], local_repo=True)
# info('human_ensembl_regulatory', local_repo=True)

In [ ]:
train_dset = get_dataset(config["dataset"], 'train', force_download=True, local_repo=True)
# train_dset = get_dataset("/home/davidcechak/genomic_benchmarks/datasets/human_ensembl_regulatory", 'train', force_download=True)
train_dset

In [ ]:
# TODO remove hack
# train_dset.all_paths = train_dset.all_paths[:103]
# train_dset.all_labels = train_dset.all_labels[:103] 
train_dset.__len__()

## Tokenizer and vocab

In [ ]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

## Dataloader and batch preparation

In [ ]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

## Model

In [ ]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

## Training

In [ ]:
model.train(train_loader, epochs=config["epochs"])

## Testing

In [ ]:
test_dset = get_dataset(config["dataset"], 'test')

# TODO remove hack
# test_dset.all_paths = test_dset.all_paths[:100]
# test_dset.all_labels = test_dset.all_labels[:100] 
# test_dset.all_paths = test_dset.all_paths[17300:21500]
# test_dset.all_labels = test_dset.all_labels[17300:21500] 
# print(test_dset.__len__())

test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

# acc, f1 = model.test_multiclass(test_loader, class_count=3)
acc = model.test_multiclass(test_loader, class_count=3)
# acc, f1
acc